## History based recommendation system using Pandas , sklearn and NLTK

In [2]:
!pip install nltk
!pip install scikit-learn
import pandas as pd
import numpy as np
import sqlite3
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import *
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import *

  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)


In [3]:
## initialization
df = pd.DataFrame()
stemmer = PorterStemmer()
con = sqlite3.Connection("db.sqlite3")
tfidf = TfidfVectorizer(lowercase=True)
nnmd = NearestNeighbors(n_neighbors=10,n_jobs=-1)

### Text and data processing

In [4]:
def fetchDataPreprocessed():
    
    global df
    df = pd.read_sql("select * from shop_product_detail",con=con)
    titles = textPreprocessing(" ".join(df['product_name'].values))
    tfidf.fit([titles])
    data = [tfidf.transform([textPreprocessing(x)]).toarray()[0] for x in df['product_name'].values]
    return csr_matrix(data)
    
def textPreprocessing(s):
    s = re.sub("[^a-zA-Z0-9]"," ",s).lower()
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    text = pattern.sub('', s)
    return text



In [5]:
d = fetchDataPreprocessed()

### Model building

In [6]:
def setupKNN(data1):
    nnmd.fit(data1)
    

In [7]:
setupKNN(d)

In [8]:
def recommendProductsUsingKNN(s):
    s = " ".join(s)
    s = textPreprocessing(s)
    s = tfidf.transform([s])
    distances , indices = nnmd.kneighbors(s.toarray(),n_neighbors=10)
    return df.loc[indices[0],:]

In [10]:
recommendProductsUsingKNN(["jeans","samsung mobile"])

,product_id,product_name,description,img1,img2,img3,brand,price,categoryid,productrating,quantity,product_specification,category
13667,13675,Samsung Pouch for Samsung Galaxy Note 510,Samsung,shop/images/unknown.png,,,Samsung,2099.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Accessories
5204,5212,Amaze Mobile Car Mobile Holder Stand for Xolo ...,Amaze Mobile,shop/images/unknown.png,,,Amaze Mobile,599.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Electronics
175,183,Lee Men's Jeans,Unknown,shop/images/unknown.png,,,Unknown,2699.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Clothing
10166,10174,DEBOCK Samsung Galaxy on5 Tempered Glass for S...,DEBOCK,shop/images/unknown.png,,,DEBOCK,499.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Accessories
4727,4735,Retina Steering Mobile Holder,Retina,shop/images/unknown.png,,,Retina,399.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Electronics
7751,7759,Pepe Jeans Wayfarer Sunglasses,Pepe Jeans,shop/images/unknown.png,,,Pepe Jeans,2199.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Others
15635,15643,D-ROCK Jeans Girl's Combo,D-ROCK,shop/images/unknown.png,,,D-ROCK,1499.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Clothing
15632,15640,D-ROCK Jeans Girl's Combo,D-ROCK,shop/images/unknown.png,,,D-ROCK,1499.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Clothing
15186,15194,Riot Jeans Women's Camisole,Unknown,shop/images/unknown.png,,,Unknown,349.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Clothing
7741,7749,Pepe Jeans Wayfarer Sunglasses,Pepe Jeans,shop/images/unknown.png,,,Pepe Jeans,1699.0,1,0.0,10,"{\n ""color"": ""Black"" ,\n ""warranty"":""1 year...",Others
